In [2]:
!pip install pyarrow
!pip install wrds

import sys
import datetime as dt
import os
import pickle
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np
import pandas as pd
import os
import sys
import pyarrow.parquet as pq
import re
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import wrds
import pytz
import math
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
import ast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 54.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
INFO: pip is looking at multiple versions of wrds to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.13
    Uninstalling SQLAlchemy-2.0.13:
      Successfully uninstalled SQLAlchemy-2.0.13

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
folder_path = r'/data/workspace_files/parquet_files/reports_dictionary'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [6]:
def jaccard_similarity(str1, str2):
    try:
        dict1 = ast.literal_eval(str1)
        dict2 = ast.literal_eval(str2)
        s1 = set(dict1.keys())
        s2 = set(dict2.keys())
        return len(s1.intersection(s2)) / len(s1.union(s2))
    except:
        return -1

def cosine_similarity(str1, str2):
    try:
        dict1 = ast.literal_eval(str1)
        dict2 = ast.literal_eval(str2)
        list1 = [dict1.get(key, 0) for key in set(dict1.keys()) | set(dict2.keys())]
        list2 = [dict2.get(key, 0) for key in set(dict1.keys()) | set(dict2.keys())]
        a = np.array(list1)
        b = np.array(list2)
        cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return cos_sim
    except:
        return -1

In [7]:
# Specify the path to the Parquet file
parquet_file_path = f'{folder_path}/file_2015.parquet'

# Read the Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_path)

# Convert the PyArrow Table to a Pandas DataFrame (optional)
df = table.to_pandas()

# Now, you can work with the PyArrow Table or Pandas DataFrame as needed
df.head()

,cik,filing_date,accession_number,conformed_period_report,form_type,company_name,total_words,doc_dict_text
0,22444,2015-01-09,0000022444-15-000006,2015-01-09,10-Q,COMMERCIAL METALS CO,16837,"{'ABILITY': 10, 'ABLE': 1, 'ABOUT': 10, 'ABOVE..."
1,1137789,2015-01-30,0001104659-15-005719,2015-01-02,10-Q,Seagate Technology plc,28710,"{'ABILITY': 8, 'ABLE': 1, 'ABOUT': 6, 'ABOVE':..."
2,1378992,2015-01-30,0001378992-15-000012,2015-01-30,10-Q,BERRY PLASTICS GROUP INC,20274,"{'ABATE': 1, 'ABILITY': 9, 'ABLE': 1, 'ABOUT':..."
3,1493594,2015-02-02,0001193125-15-029663,2015-01-02,10-Q,M/A-COM Technology Solutions Holdings Inc.,42425,"{'ABANDONED': 2, 'ABIDE': 1, 'ABILITY': 51, 'A..."
4,67887,2015-02-03,0000067887-15-000009,2015-01-03,10-Q,MOOG INC.,30487,"{'ABANDON': 1, 'ABILITY': 11, 'ABLE': 3, 'ABOU..."


In [8]:

def process_and_save(year):
    folder_path = r'/data/workspace_files/parquet_files'
    curr_file_path = f'{folder_path}/file_{str(year)}.parquet'
    prev_file_path = f'{folder_path}/file_{str(year-1)}.parquet'
    
    # Read only relevant columns from current Parquet file
    table_curr = pq.read_table(curr_file_path, columns=['cik', 'form_type', 'filing_date', 'conformed_period_report', 'doc_dict_text'])
    curr_df = table_curr.to_pandas()
    curr_df['conformed_period_report'] = pd.to_datetime(curr_df['conformed_period_report'])
    curr_df['month'] = curr_df['conformed_period_report'].dt.month
    
    # Read only relevant columns from previous Parquet file
    table_prev = pq.read_table(prev_file_path, columns=['cik', 'form_type', 'conformed_period_report', 'doc_dict_text'])
    prev_df = table_prev.to_pandas()
    prev_df['conformed_period_report'] = pd.to_datetime(prev_df['conformed_period_report'])
    prev_df['month'] = prev_df['conformed_period_report'].dt.month
    
    # Merge DataFrames on 'cik', 'form_type', and 'month'
    df = pd.merge(curr_df, prev_df, on=['cik', 'form_type', 'month'], suffixes=['', '_prev'])
    
    df['cosine_similarity'] = df.apply(lambda x: cosine_similarity(x['doc_dict_text'], x['doc_dict_text_prev']), axis=1)
    df['jaccard_similarity'] = df.apply(lambda x: jaccard_similarity(x['doc_dict_text'], x['doc_dict_text_prev']), axis=1)
    
    # Select and save only necessary columns
    df = df[['cik', 'filing_date', 'cosine_similarity', 'jaccard_similarity']]
    df.to_parquet(f'/data/workspace_files/parquet_files/similarity_scores/by_conformed_period/{str(year)}.parquet')
    # Release memory
    del curr_df, prev_df, table_curr, table_prev, df


dfs = []


In [16]:
for year in range(2015, 2022):
    process_and_save(year)
    print(f'Finished: {year}')

final_df = pd.concat(dfs)
print(len(final_df))
print(len(final_df['cik'].unique()))

final_df['filing_monthyr'] = final_df['filing_date'].dt.strftime('%b%Y')

Finished: 2015
Finished: 2016
Finished: 2017
Finished: 2018
Finished: 2019


<ipython-input-5-580382473648>:22: RuntimeWarning: invalid value encountered in scalar divide
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


FileNotFoundError: FileNotFoundError: /data/workspace_files/parquet_files/file_2020.parquet

In [14]:
final_df[final_df['filing_monthyr']=='Feb2017']

,cik,filing_date,cosine_similarity,jaccard_similarity,filing_monthyr
29482,1373690,2017-02-28,0.992972,0.774209,Feb2017


In [10]:
month_yrs = final_df['filing_monthyr'].unique()
for month_yr in month_yrs:
    df_filt = final_df[final_df['filing_monthyr']==month_yr]
    df_filt.to_parquet(f'/data/workspace_files/parquet_files/similarity_scores/{month_yr}.parquet')